# Opinion dynamics over discourse sheaves

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

## Constant sheaves with stalk equal to $\mathbb{R}$ on each vertex and node

### Expression diffusion for "learning to lie" dynamics

In [3]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,3),
    (0,6),
    (1,2),
    (1,5),
    (2,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

d = 1                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(d,d),
        e[1]:np.random.randn(d,d)
        } 
        for e in edges
    }                                           # Incidency linear maps

# Sheaf representation 

# Coboundary map

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = 1
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - 1

# Sheaf Laplacian

L_f = B.T @ B

In [4]:
B

array([[ 1., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  1.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.]])

In [18]:
def expressionDynamic(
        t, 
        B_flatten, 
        P,
        E,
        V, 
        d, 
        beta, 
        x0
        ):
    
    B = B_flatten.reshape(E*d,V*d)
    dt_dB = (- beta * P * (B @ np.outer(x0, x0)))
    return dt_dB.flatten()

def expressionDynamicSolver(
        B0, 
        x0, 
        E,
        V,
        d,
        beta = 1,
        T = 100,
        time_span = 100
        ):

    time_points = np.linspace(0, T, time_span)
    P = (B0 != 0).astype('int32')

    solution = solve_ivp(
        expressionDynamic, 
        [0, 100], 
        B0.flatten(), 
        t_eval=time_points, 
        args=(P, E, V, d, beta, x0,),
        method='RK45'
        )
    
    return solution.y[:,-1].reshape(E*d, V*d)

In [19]:
B0 = np.copy(B)
x0 = np.random.randn(V)

B_hat = expressionDynamicSolver(B0, x0, E, V, d)

In [20]:
B_hat

array([[ 0.81303881,  0.13403998,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-0.00414487,  0.        ,  0.        , -0.00417993,  0.        ,
         0.        ,  0.        ],
       [ 0.09267787,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -1.07771505],
       [ 0.        ,  0.32806281, -1.18588454,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.56398433,  0.        ,  0.        ,  0.        ,
        -1.20386829,  0.        ],
       [ 0.        ,  0.        ,  0.02515894,  0.        , -1.02395226,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.96079301,
         0.        , -0.91854778],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.02926207, -0.96891541]])

In [21]:
L_hat = B_hat.T @ B_hat

In [22]:
x0.T @ L_f @ x0

7.4222220020796

In [23]:
x0.T @ L_hat @ x0

0.0010892996321604416